In [152]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import time
from datetime import date
import pandas as pd


In [195]:
# The input
datain = np.loadtxt('04a_input.txt',comments=None,dtype=str,usecols=(0,1,2,3))

year = np.empty((datain.shape[0],1))
month = np.empty_like(year)
day = np.empty_like(year)
hour = np.empty_like(year)
minute = np.empty_like(year)

for i in range(datain.shape[0]):
    year[i], month[i], day[i] = datain[i,0].replace('[','').split('-') # remove open bracket, split by '-'
    hour[i], minute[i] = datain[i,1].replace(']','').split(':') # remove close bracket
    if datain[i,3][0] == '#':
        datain[i,3] = datain[i,3].replace('#','') # 
    else:
        datain[i,3] = ''
    

In [198]:
year[:5],month[:5],day[:5],hour[:5],minute[:5],datain[:5,3]

(array([[1518.],
        [1518.],
        [1518.],
        [1518.],
        [1518.]]), array([[4.],
        [5.],
        [6.],
        [4.],
        [4.]]), array([[25.],
        [22.],
        [25.],
        [13.],
        [10.]]), array([[ 0.],
        [23.],
        [ 0.],
        [ 0.],
        [ 0.]]), array([[24.],
        [50.],
        [25.],
        [54.],
        [ 4.]]), array(['', '3299', '', '', '2467'], dtype='<U11'))

In [233]:
dfin = pd.DataFrame(np.squeeze(np.array((year,month,day,hour,minute))).T)
dfin = dfin.rename(columns={0: "year", 1: "month", 2: "day", 3: "hour", 4: "minute"})
#dfin = dfin.rename(['year','month','day','hour','minute'],axis='columns')
dfin[:][:5]

,year,month,day,hour,minute
0,1518.0,4.0,25.0,0.0,24.0
1,1518.0,5.0,22.0,23.0,50.0
2,1518.0,6.0,25.0,0.0,25.0
3,1518.0,4.0,13.0,0.0,54.0
4,1518.0,4.0,10.0,0.0,4.0


In [235]:
pd.to_datetime(dfin)
dfin[:][:5]


ValueError: cannot assemble the datetimes: 'float' object is unsliceable

In [150]:
datestimes = np.empty_like(datain).astype('datetime64[m]')
datestimes = np.delete(datestimes,1,axis=1) # specify the columns to be removed
for i in range(len(datestimes)):
    temp = str(datain[i,0]) + ' ' + str(datain[i,1])
    datestimes[i] = np.datetime64(temp)
    

    

In [151]:
datain[:5]

array([['1518-04-25', '00:24', 'falls', ''],
       ['1518-05-22', '23:50', 'Guard', '3299'],
       ['1518-06-25', '00:25', 'falls', ''],
       ['1518-04-13', '00:54', 'wakes', ''],
       ['1518-04-10', '00:04', 'Guard', '2467']], dtype='<U11')

In [179]:
dfin = pd.DataFrame(np.append(datain[:,:2],datain[:,2:],axis=1))

In [180]:
dfin[:2]

,0,1,2,3
0,1518-04-25,00:24,falls,
1,1518-05-22,23:50,Guard,3299


In [183]:
dfin['timestamp'] = pd.to_datetime(dfin[0] + ' ' + dfin[1],unit='ns')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1518-04-25 00:24:00

In [185]:
year = np.empty((datain.shape[0],1))
year.shape

(1072, 1)

In [10]:
datain[0]

array(['[1518-04-25', '00:24]'], dtype='<U11')

In [11]:
str(datain[0])


"['[1518-04-25' '00:24]']"

In [13]:
date.today()


datetime.date(2018, 12, 3)

In [186]:
test = datain[:5]


for entry in test:
    entry[0] = entry[0].replace('[','') # remove open bracket
    entry[1] = entry[1].replace(']','') # remove close bracket



In [194]:
test[0,1].split(':')

['00', '24']

In [121]:
new = np.empty_like(test).astype('datetime64[m]')
new = np.delete(new,1,axis=1)
for i in range(len(new)):
    temp = str(test[2,0]) + ' ' + str(test[2,1])
    new[i] = np.datetime64(temp)
    
new

array([['1518-06-25T00:25'],
       ['1518-06-25T00:25'],
       ['1518-06-25T00:25'],
       ['1518-06-25T00:25'],
       ['1518-06-25T00:25']], dtype='datetime64[m]')

In [173]:
testdf = pd.DataFrame(new)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1518-06-25 00:25:00

In [115]:
new = np.empty((10,1),dtype='datetime64[m]')
new[0] = np.datetime64(temp)
new

array([[          '1518-06-25T00:25'],
       ['-8768294034267-12-19T20:23'],
       [         '10300-05-10T19:49'],
       [         '10300-11-28T11:20'],
       [         '10300-09-20T19:20'],
       [         '10300-11-28T12:24'],
       [         '10231-01-13T18:24'],
       [          '1970-01-01T00:00'],
       [         '10173-09-21T14:48'],
       [    '2700376427-02-26T14:41']], dtype='datetime64[m]')

In [120]:
new

array([['NaT'],
       ['NaT'],
       ['NaT'],
       ['NaT'],
       ['NaT']], dtype='datetime64[m]')

In [86]:
temp

'1518-06-25 00:25'

In [87]:
new[2] = temp

In [88]:
new[2]

array(['1518-06-25 '], dtype='<U11')